In [7]:
import talib as taLib
import pymongo
import pandas as pd
import numpy as np
import nsepy
import math
from datetime import datetime
from datetime import timedelta
from datetime import date
from pymongo import MongoClient
from nsepy import get_history
from TAcharts.indicators.ichimoku import Ichimoku
client=MongoClient("mongodb://localhost:27018/",username="stockadm",password="1qaz@WSX")
db=client["stockWarehouse"]
equityList=db["stocklist"]
stockWarehouse=db["stockWarehouse"]
equityListPd = pd.DataFrame(equityList.find({}))
equityListPd.shape[0]

8425

In [2]:
print(equityListPd.columns)
equityListPd.head(5)

Index(['Face Value', 'Group', 'ISIN No', 'Industry', 'Issuer Name',
       'Security Code', 'Security Id', 'Security Name', 'State', 'Status',
       '_id', 'optionsTrade'],
      dtype='object')


,Face Value,Group,ISIN No,Industry,Issuer Name,Security Code,Security Id,Security Name,State,Status,_id,optionsTrade
0,INE117A01022,2,Heavy Electrical Equipment,Equity,ABB,500002,ABB India Limited,Active,A,B,5e6dffa36cef0bbb3f7833ea,NaN
1,INE424A01014,10,,Equity,TPAEC,500004,TORRENT POWER AEC LTD.,Delisted,NaN,B,5e6dffa36cef0bbb3f7833eb,NaN
2,INE885A01032,1,Auto Parts & Equipment,Equity,AMARAJABAT,500008,AMARA RAJA BATTERIES LTD.,Active,A,A,5e6dffa36cef0bbb3f7833ec,A
3,INE984C01013,10,Iron & Steel Products,Equity,AKARLAMIN,500005,AKAR LAMINATORS LTD.,Delisted,NaN,XD,5e6dffa36cef0bbb3f7833ed,NaN
4,INE001A01036,2,Housing Finance,Equity,HDFC,500010,HOUSING DEVELOPMENT FINANCE CORP.LTD.,Active,A,A,5e6dffa36cef0bbb3f7833ee,A


In [3]:
def dataCollector():
    equityListPd = pd.DataFrame(equityList.find({'Security Name':'Active','State':"A"}))
    collectionTime=2000
    basicSample=30
    cnt=0
    my_time = datetime.now()

    for rows in equityListPd['Issuer Name'].iteritems():
        print(cnt,rows[1],end="-")
        cnt= cnt + 1
        if stockWarehouse.find({'Symbol':rows[1]}).count() > 1 :
            for j in stockWarehouse.find({'Symbol':rows[1]},{'_id':0,'Date':1}).sort('Date',-1).limit(1) :
                collectionTime=((my_time - j['Date']).days - 1)
                fromDate=j['Date']
        iters=int(collectionTime/basicSample)
        if (collectionTime != 0):
            if collectionTime < 2000:
                basicSample=collectionTime
                iters=1
            dt=datetime.now()
            print("Iterations:",iters,end="|")
            for i in range(0,iters):

                startDate=dt - timedelta(days=basicSample)
                if( startDate <= fromDate):
                    startDate = (fromDate + timedelta(days=1))
                endDate=dt 
                print(i,"Collecting into between",startDate.date(),"-", endDate.date(),"@",datetime.now(),end="|")
                df=pd.DataFrame(get_history(symbol=rows[1] , start= startDate,end=endDate))
                df.reset_index(inplace=True)
                df['Date']=pd.to_datetime(df['Date'])
                dfLength=(len(df.index))
                if  dfLength > 0 :
                    data_dict=df.to_dict("records")
                    stockWarehouse.insert_many(data_dict)
                    tmpdf=pd.DataFrame(stockWarehouse.find({"Symbol":rows[i],"Date":{"$lte":endDate,"$gte":startDate}}))
                    print(df.index.size,tmpdf.index.size,end="|")

                else :
                    print("Marking as Inactive")
                    equityList.update_many({'Issuer Name':rows[1]},{'$set': {'Security Name':'Inactive'}})
                    break;
                dt=(startDate - timedelta(days=1))


In [10]:
def checkType(x):
    if x['Close']>x['Open']:
        return 1
    else :
        return -1
def pullbackTrendIdentify(df):
    df['type']=df.apply(lambda x: checkType(x),axis=1)
    length=df.shape[0] -1
    uptrenddaycnt=0
    bearcnt=0
    slope=0
    stoploss=0
    target=0
    signal=0
                
    if df['type'].iloc[length] > 0:
        for i in range(length-1,1,-1):
            if df['type'].iloc[i] < 0 :
                bearcnt=bearcnt+1
            else:
                break;
        if bearcnt > 2:
            (macd,macdsignal,macdhistory)=(taLib.MACD(datadf['Close'], fastperiod=26, slowperiod=9, signalperiod=9))
            df['macd']=macd
            for i in range((length - 1) - bearcnt,1,-1):
                if df['macd'].iloc[i] >0:
                    uptrenddaycnt=uptrenddaycnt+1
                else:
                    break;
            if uptrenddaycnt > 1:
                high=df['High'].iloc[(length-uptrenddaycnt):length].max()
                low=df['Low'].iloc[(length-uptrenddaycnt):length].min()
                slope=math.degrees(math.atan((high-low)/uptrenddaycnt))
                signal=1
            if signal==1:
                stoploss=df['Low'].iloc[(length -1) - bearcnt - uptrenddaycnt: length - 1].min()
                target=df['Close'].iloc[length] *1.05
    return (df,slope,bearcnt,uptrenddaycnt,stoploss,target,signal,df['Close'].iloc[length])
    



In [15]:
equityList=db["stocklist"]
stockWarehouse=db["stockWarehouse"]
equityListdf=pd.DataFrame(equityList.find({'State':"A"},{'Issuer Name':1,'_id':0}))
for symbol in equityListdf['Issuer Name']:

    datadf=pd.DataFrame(stockWarehouse.find({'Symbol':symbol},{'_id':0,'%Deliverable':1,'Close':1,'Open':1,'High':1,'Low':1,'Date':1}).sort('Date',-1).limit(90))
    datadf.set_index('Date', inplace=True)
    datadf.sort_index(ascending=True,inplace=True)
    (datadf,slope,bearcnt,uptrenddaycnt,stoploss,target,signal,close)=pullbackTrendIdentify(datadf)
    if signal > 0 and slope > 30 and slope < 50:
        print(symbol,slope,bearcnt,uptrenddaycnt,stoploss,target,signal,close)
        print(datadf.tail(5))

EXIDEIND 43.98781238601755 3 36 161.3 206.1675 1 196.35
             Close    High     Low    Open  type      macd
Date                                                      
2020-12-29  191.35  193.15  184.10  187.65     1  3.419948
2020-12-30  191.35  193.50  187.75  192.90    -1  3.642672
2020-12-31  191.25  193.25  190.55  192.35    -1  3.741095
2021-01-01  191.35  192.90  190.50  191.70    -1  3.771169
2021-01-04  196.35  197.75  190.90  192.35     1  4.367207
RELCAPITAL 34.5085229876684 13 4 8.2 14.5425 1 13.85
            Close   High    Low   Open  type      macd
Date                                                  
2020-12-10  11.95  11.95  11.00  11.95    -1  0.945136
2020-12-11  12.50  12.50  12.50  12.50    -1  1.118296
2020-12-14  13.10  13.10  13.10  13.10    -1  1.305552
2020-12-15  13.20  13.75  12.45  13.75    -1  1.437510
2020-12-16  13.85  13.85  13.25  13.50     1  1.595813
HINDZINC 44.45778483505365 6 32 215.2 256.2525 1 244.05
             Close    High    Low    

In [13]:
datadf.tail(10)

,Close,High,Low,Open,type,macd
Date,,,,,,
2020-12-21,18.90,20.90,18.00,20.90,-1,1.152678
2020-12-22,19.05,19.25,17.80,18.35,1,0.955425
2020-12-23,20.40,20.90,19.20,19.40,1,0.965158
2020-12-24,20.05,21.15,19.60,20.55,-1,0.913995
2020-12-28,19.90,20.35,19.75,20.15,-1,0.843666
2020-12-29,21.25,21.65,19.60,20.00,1,0.949072
2020-12-30,20.65,21.30,20.40,21.30,-1,0.937535
2020-12-31,20.70,20.90,20.45,20.75,-1,0.921396
2021-01-01,20.70,20.90,20.50,20.75,-1,0.895790
